In [1]:
import numpy as np
from keras.utils import to_categorical
from keras import models
from keras.callbacks import ModelCheckpoint
from keras import layers
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import pickle
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"


Using TensorFlow backend.


In [ ]:
DATA = "ARTINT"

In [2]:
!nvidia-smi

Sun Jan 13 12:33:08 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0  On |                  N/A |
|  0%   35C    P8    18W / 250W |    272MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 23%   35C    P8    16W / 250W |    506MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [3]:
d_feature_vector = pickle.load(open(DATA+"_train_only_bib_feature_vector.p", "rb"))
d_feature_vector_test = pickle.load(open(DATA+"_test_only_bib_feature_vector.p", "rb"))



In [4]:

x_train = []
y_train = []
for i in d_feature_vector.keys():
    x_train.append(d_feature_vector[i])
    label = 1.0
    if i.startswith("neg_"):
        label = 0.0
    y_train.append(label)
x_train = np.array(x_train)
y_train = np.array(y_train)

print(x_train.shape)


(3056, 9000)


In [5]:

x_test = []
y_test = []
for i in d_feature_vector_test.keys():
    x_test.append(d_feature_vector_test[i])
    label = 1.0
    if i.startswith("neg_"):
        label = 0.0
    y_test.append(label)
x_test = np.array(x_test)
y_test = np.array(y_test)

print(x_test.shape)


(768, 9000)


In [6]:
c = 0
for i in y_test:
    if i == 0.0:
        c = c+ 1
print(c)

364


In [7]:
# Input - Layer
model = models.Sequential()
model.add(layers.Dense(50, activation = "relu", input_dim=9000))
# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                450050    
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 455,201
Trainable params: 455,201
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile( optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
checkpoint = ModelCheckpoint(DATA+'-only-bib-model-best.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  


In [9]:
results = model.fit( x_train,y_train, epochs= 10, batch_size = 32, callbacks = [checkpoint],validation_data = (x_test, y_test))


Train on 3056 samples, validate on 768 samples
Epoch 1/10
3056/3056 [==============================] - 2s 764us/step - loss: 0.4367 - acc: 0.7968 - val_loss: 0.2969 - val_acc: 0.8659

Epoch 00001: val_loss improved from inf to 0.29692, saving model to JNCA-only-bib-model-best.h5
Epoch 2/10
3056/3056 [==============================] - 1s 168us/step - loss: 0.2102 - acc: 0.9126 - val_loss: 0.3003 - val_acc: 0.8763

Epoch 00002: val_loss did not improve from 0.29692
Epoch 3/10
3056/3056 [==============================] - 1s 175us/step - loss: 0.1317 - acc: 0.9496 - val_loss: 0.3386 - val_acc: 0.8776

Epoch 00003: val_loss did not improve from 0.29692
Epoch 4/10
3056/3056 [==============================] - 0s 162us/step - loss: 0.0942 - acc: 0.9617 - val_loss: 0.4460 - val_acc: 0.8815

Epoch 00004: val_loss did not improve from 0.29692
Epoch 5/10
3056/3056 [==============================] - 1s 164us/step - loss: 0.0779 - acc: 0.9627 - val_loss: 0.5358 - val_acc: 0.8789

Epoch 00005: val_lo

In [10]:
model.load_weights(DATA+'-only-bib-model-best.h5', by_name = True)
predicted =  model.predict(x_test)
for i in range(len(predicted)):
    if predicted[i] >= 0.5:
        predicted[i] = 1.0
    else :
        predicted[i] = 0.0

In [11]:
print("fscore",f1_score(y_test, predicted, average="macro"))
print("precision",precision_score(y_test, predicted, average="macro"))
print("recall",recall_score(y_test, predicted, average="macro"))    

x = classification_report(y_test,predicted)
print(x)

fscore 0.865883370209303
precision 0.8677590764233829
recall 0.868036666303993
             precision    recall  f1-score   support

        0.0       0.83      0.91      0.87       364
        1.0       0.91      0.83      0.87       404

avg / total       0.87      0.87      0.87       768



In [ ]:
model.save(DATA+'only_bib_model')